In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Zillow ZRI

In [42]:
main_dataframe = pd.read_csv('../../data/zillow/Zip_Zri_MultiFamilyResidenceRental.csv', dtype = {'RegionName':str})
main_dataframe.rename(columns = {'RegionName':'Zipcode'}, inplace = True)
main_dataframe['Zipcode'] = main_dataframe['Zipcode'].str.zfill(5)

In [43]:
main_dataframe = pd.melt(main_dataframe, id_vars =main_dataframe.columns[:7],
                         value_vars = main_dataframe.columns[7:],
                         var_name = 'Date', 
                         value_name = 'Rent')
main_dataframe['Date'] = pd.to_datetime(main_dataframe['Date'])

In [44]:
fill_rents = main_dataframe[['Zipcode', 'Rent', 'Date']].copy()
# zori_sf_metro.drop('Rent', axis = 1, inplace = True)

fill_rents = fill_rents.reset_index().pivot(index = 'Date',columns = 'Zipcode')['Rent'].reset_index()
for code in fill_rents.columns[1:]:
     fill_rents[code].interpolate(inplace = True)
fill_rents.fillna(method = 'bfill',inplace = True)
fill_rents = pd.melt(fill_rents, id_vars='Date', 
                                  value_vars = fill_rents.columns[1:],
                                  var_name='Zipcode',value_name = 'Rent')

main_dataframe.drop('Rent', axis = 1, inplace = True)
main_dataframe = pd.merge(main_dataframe, fill_rents, on = ['Date','Zipcode'])

In [47]:
main_dataframe.isnull().sum()

RegionID        0
Zipcode         0
City            0
State           0
Metro         339
CountyName      0
SizeRank        0
Date            0
Rent            0
dtype: int64

In [117]:
main_dataframe['Year'] = main_dataframe['Date'].dt.year
main_dataframe

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,Date,Rent,Year
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,2010-09-01,2930.0,2010
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,2010-09-01,1447.0,2010
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,2010-09-01,2797.0,2010
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,2010-09-01,1081.0,2010
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,2010-09-01,1437.0,2010
...,...,...,...,...,...,...,...,...,...,...
210288,90755,75202,Dallas,TX,Dallas-Fort Worth-Arlington,Dallas County,1857,2020-01-01,1739.0,2020
210289,94610,84627,Ephraim,UT,NaN,Sanpete County,1858,2020-01-01,1298.0,2020
210290,84452,60301,Oak Park,IL,Chicago-Naperville-Elgin,Cook County,1859,2020-01-01,1625.0,2020
210291,94629,84647,Mount Pleasant,UT,NaN,Sanpete County,1860,2020-01-01,1221.0,2020


## Air Quality

In [123]:
fpath = '../../data/airqual/'
cols = ['Date Local','Arithmetic Mean','State Name', 'County Name', 'City Name']
file_list = [(f'{fpath}/daily_42602_{year}.csv') for year in range(2010, 2021)]
dataframe = pd.read_csv(file_list[0], usecols = cols)

In [191]:
file_list = [(f'{fpath}/daily_42602_{year}.csv') for year in range(2010, 2021)]
cols = ['Date Local', 'Arithmetic Mean', 'State Name', 'County Name', 'City Name']

dataframe = pd.read_csv(file_list[0], usecols = cols)

for file in file_list[1:]:
    new_df = pd.read_csv(file, usecols = cols)
    dataframe = pd.concat([dataframe, new_df], axis=0)

In [192]:
dataframe['Date Local'] = pd.to_datetime(dataframe['Date Local'])
dataframe['Month'] = dataframe['Date Local'].dt.month
dataframe['Year'] = dataframe['Date Local'].dt.year
dataframe = dataframe.groupby(['State Name', 'County Name', 'City Name', 'Year', 'Month'])[['Arithmetic Mean']].agg('mean').reset_index()
dataframe.Month = dataframe.Month.astype(str)
dataframe.Month = dataframe.Month.str.zfill(2)
dataframe['Date'] = '01/'+dataframe.Month.astype(str)+'/'+dataframe.Year.astype(str)
dataframe['Date'] = pd.to_datetime(dataframe['Date'], format="%d/%m/%Y")
dataframe.drop(['Month', 'Year'],axis=1,inplace=True)

dataframe.rename(columns={'State Name': 'State', 
                            'County Name': 'County', 
                            'City Name':'City',
                            'Arithmetic Mean':'AQI_mean_ppb'}, inplace=True)

In [189]:
dataframe['Date'] = pd.to_datetime(dataframe['Date'], format="%d/%m/%Y")

In [193]:
dataframe

,State,County,City,AQI_mean_ppb,Date
0,Alabama,Jefferson,Birmingham,12.065635,2013-12-01
1,Alabama,Jefferson,Birmingham,12.899658,2014-01-01
2,Alabama,Jefferson,Birmingham,8.851034,2014-02-01
3,Alabama,Jefferson,Birmingham,13.212881,2014-03-01
4,Alabama,Jefferson,Birmingham,11.379411,2014-04-01
...,...,...,...,...,...
42884,Wyoming,Weston,Newcastle,2.548553,2016-06-01
42885,Wyoming,Weston,Newcastle,2.558559,2016-07-01
42886,Wyoming,Weston,Newcastle,1.258537,2016-08-01
42887,Wyoming,Weston,Newcastle,0.539661,2016-09-01


## Personal Income

In [95]:
path = '../../data/real_personal_income.csv'
dataframe = pd.read_csv(path)
dataframe = dataframe[dataframe.LineCode == 2]
dataframe.drop(['GeoFips','LineCode'],axis=1,inplace=True)

dataframe = dataframe.melt(id_vars = ['MetroArea','Description'],
                            var_name='Year', 
                            value_name='Personal Income').\
                            drop('Description',axis=1)

dataframe['Year'] = pd.to_datetime(dataframe['Year']).dt.year

dataframe.rename(columns={'MetroArea':'City'}, inplace=True)

In [96]:
dataframe

,City,Year,Personal Income
0,Austin,2008,44521
1,Miami,2008,44447
2,New York,2008,48585
3,San Francisco,2008,54862
4,Austin,2009,42103
5,Miami,2009,40469
6,New York,2009,46606
7,San Francisco,2009,52251
8,Austin,2010,43390
9,Miami,2010,42729


##  Income Level CRC

In [225]:
dataframe = pd.read_csv('../../data/volume_data_Income_Level_CRC.csv')

In [226]:
dataframe

,month,date,vol,vol_unadj,income_level_group
0,60,2005-01,1.461366e+10,1.287236e+10,High
1,60,2005-01,1.527169e+10,1.342024e+10,Middle
2,60,2005-01,4.578106e+09,3.978007e+09,Moderate
3,60,2005-01,7.635135e+08,6.517910e+08,Low
4,61,2005-02,1.433436e+10,1.272458e+10,High
...,...,...,...,...,...
647,221,2018-06,8.982546e+08,9.167395e+08,Low
648,222,2018-07,1.279173e+10,1.364601e+10,Middle
649,222,2018-07,1.368489e+10,1.400351e+10,High
650,222,2018-07,4.220966e+09,4.625639e+09,Moderate


In [227]:
dataframe.drop('month', axis=1, inplace=True)
dataframe = dataframe[(dataframe['income_level_group'] != "High") & 
                        (dataframe['income_level_group'] != "Middle")]

dataframe.rename(columns={'date':'Date'}, inplace=True)

dataframe['Date'] = pd.to_datetime(dataframe['Date']+'-01', format="%Y-%m-%d")

dataframe

C:\Users\eugen\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-227-805efe9c4044>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Date'] = pd.to_datetime(dataframe['Date']+'-01', format="%Y-%m-%d")


,Date,vol,vol_unadj,income_level_group
2,2005-01-01,4.578106e+09,3.978007e+09,Moderate
3,2005-01-01,7.635135e+08,6.517910e+08,Low
6,2005-02-01,4.563908e+09,3.900504e+09,Moderate
7,2005-02-01,7.558710e+08,6.549358e+08,Low
10,2005-03-01,4.762458e+09,4.775794e+09,Moderate
...,...,...,...,...
643,2018-05-01,8.904970e+08,9.429447e+08,Low
646,2018-06-01,4.398129e+09,4.513570e+09,Moderate
647,2018-06-01,8.982546e+08,9.167395e+08,Low
650,2018-07-01,4.220966e+09,4.625639e+09,Moderate


## Census

In [218]:
dataframe = pd.read_csv('../../data/census-query.csv',dtype={'zip_code':str})
dataframe

,geo_id,do_date,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,...,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english,zip_code
0,87537,2014-01-01,2510,856,1283,1227,42.1,58,75,87,...,NaN,304,171,80,593,20,NaN,NaN,NaN,87537
1,87017,2014-01-01,346,112,150,196,50.9,0,0,12,...,NaN,0,27,15,96,40,NaN,NaN,NaN,87017
2,87528,2014-01-01,3505,727,1758,1747,27.9,260,194,159,...,NaN,293,266,298,1237,138,NaN,NaN,NaN,87528
3,87533,2014-01-01,133,58,49,84,25.8,0,12,8,...,NaN,24,8,7,58,3,NaN,NaN,NaN,87533
4,87511,2014-01-01,2896,787,1177,1719,36.0,142,63,44,...,NaN,263,107,163,1008,279,NaN,NaN,NaN,87511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33084,35463,2014-01-01,1794,704,904,890,42.6,84,46,58,...,217.0,134,50,113,389,50,NaN,NaN,NaN,35463
33085,77664,2014-01-01,2667,934,1300,1367,45.6,29,54,113,...,212.0,38,286,125,647,171,NaN,NaN,NaN,77664
33086,21822,2014-01-01,2013,776,1070,943,36.8,34,59,88,...,106.0,83,198,40,645,240,NaN,NaN,NaN,21822
33087,7418,2014-01-01,2304,766,1281,1023,40.8,56,27,93,...,336.0,81,93,292,711,136,NaN,NaN,NaN,07418


# Transformer Function Testing

In [243]:
from transformers import *

zillow_path = '../../data/zillow/Zip_Zri_MultiFamilyResidenceRental.csv'
airqual_path = '../../data/airqual/' #path to folder
persinc_path = '../../data/real_personal_income.csv'
inclvl_path = '../../data/volume_data_Income_Level_CRC.csv'
census_path = '../../data/census-query.csv'

zillow_data = transform_zillow(zillow_path)
airqual_data = transform_air_qual(airqual_path)
persinc_data = transform_pers_income(persinc_path)
inclvl_data = transform_income_level(inclvl_path)
census_data = transform_census(census_path)

In [247]:
zillow_data.head()

,RegionID,Zipcode,City,State,Metro,County,SizeRank,Date,Rent,Year
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,2010-09-01,2930.0,2010
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,2010-09-01,1447.0,2010
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,2010-09-01,2797.0,2010
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,2010-09-01,1081.0,2010
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,2010-09-01,1437.0,2010


In [248]:
airqual_data.head()

,State,County,City,AQIMean,Date
0,Alabama,Jefferson,Birmingham,12.065635,2013-12-01
1,Alabama,Jefferson,Birmingham,12.899658,2014-01-01
2,Alabama,Jefferson,Birmingham,8.851034,2014-02-01
3,Alabama,Jefferson,Birmingham,13.212881,2014-03-01
4,Alabama,Jefferson,Birmingham,11.379411,2014-04-01


In [249]:
persinc_data.head()

,City,Year,PersonalIncome
0,Austin,2008,44521
1,Miami,2008,44447
2,New York,2008,48585
3,San Francisco,2008,54862
4,Austin,2009,42103


In [250]:
inclvl_data.head()

,Date,Vol,VolUnadjusted,income_level_group
2,2005-01-01,4.578106e+09,3.978007e+09,Moderate
3,2005-01-01,7.635135e+08,6.517910e+08,Low
6,2005-02-01,4.563908e+09,3.900504e+09,Moderate
7,2005-02-01,7.558710e+08,6.549358e+08,Low
10,2005-03-01,4.762458e+09,4.775794e+09,Moderate


In [251]:
census_data.head()

,geo_id,do_date,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,...,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english,Zipcode
0,87537,2014-01-01,2510,856,1283,1227,42.1,58,75,87,...,NaN,304,171,80,593,20,NaN,NaN,NaN,87537
1,87017,2014-01-01,346,112,150,196,50.9,0,0,12,...,NaN,0,27,15,96,40,NaN,NaN,NaN,87017
2,87528,2014-01-01,3505,727,1758,1747,27.9,260,194,159,...,NaN,293,266,298,1237,138,NaN,NaN,NaN,87528
3,87533,2014-01-01,133,58,49,84,25.8,0,12,8,...,NaN,24,8,7,58,3,NaN,NaN,NaN,87533
4,87511,2014-01-01,2896,787,1177,1719,36.0,142,63,44,...,NaN,263,107,163,1008,279,NaN,NaN,NaN,87511
